# Import desired utlities:

### pip - conda installs if needed:
pip install beautifulsoup4
* User friendly and extendable html parser that makes it easy to find and loop through elements
pip install lxml
* beautiful soup uses external HTML/xml parsers so far this is the only one I've used
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

pip install requests
* Used to fetch and get information about a web page request. One of the most userfriendly ways to get web page data

pip install dateparser
* You will likely run into mutiple date time formats I've had good luck with this parser that is able to take a lot for formats of a string date and convert it to a python date time.





In [1]:
import requests

from bs4 import BeautifulSoup
import dateparser

import json
import time


In [2]:
#The business page you want to pull reviews from:
#Note* it seems like the business alias is the last part of the URL may be useful in scripting
BaseUrl = "https://www.yelp.com/biz/enamel-dentistry-south-lamar-austin"

In [3]:
response = requests.get(BaseUrl)

# Check For Valid webpage response
* most webpage results will respond with status codes.
* for most people 200 is good and means you got a good response everything else is "BAD" and an error
* https://developer.mozilla.org/en-US/docs/Web/HTTP/Status
* You will also notice that the text response on yelp is "nasty" to look at
* If you do enough webscraping you can often tell what "framework" websites are using 
    * in this case you see long lines, and generic and nonhuman readable parts of the CSS: list__09f24__ynIEd
    * This usually means that a framework is being used, and potentially the CSS classes are generated dynamically with javascript or a backend language.

In [4]:
if response.status_code == 200:
    print(response.text)

<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(no-j/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>
            window.yelp = window.yelp || {};
            

In [5]:
#Parse the HTML content using lxml and put it into a BeautifulSoup object
SoupFullPage = BeautifulSoup(response.text, 'lxml')

# Crash Course HTML parsing
* Pretty much all webpages have "errors" and are not valid HTML if you were to trust a XML parser that relies on well opened closed, and structured XML.
* id's **should** only apprear once per name per page. These are great ways to select exactly what you want
* CSS classes are the next easiest thing. But these are usually not unique, so you can get odd results.
* NOTE* a css class is defined without any spaces, if there are mutiple classes listed (spaces between classes) then you can use 1 or more classes to define what you are looking for:
    * ul class="undefined list__09f24__ynIEd" there are 2 classes here a undefined one and one labeld list__09f24__ynIEd
* If you can get close to the element you want the beautifulsoup find and find_all can return just a small part of the overall page that you can then again apply another find/find_all to. 
* HTML is a hiearchy. you can navigate that hiearchy using parents or at the same level using siblings
    * https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-parents-and-find-parent
* If you can avoid it XPATH is one of the harder ways to navigate an HTML page and is not built into beautiful soup
    * /html/body/yelp-react-root/div[1]/div[4]/div/div/div[2]/div/div[1]/main/div[3]/div/section/div[2]/div/div[5]
    

In [6]:

#Note* find returns the first occurance
#find_all returns a list of elements meeting the criteria even if only 1 is found
SoupIDreviews = SoupFullPage.find("div", {"id": "reviews"})
ListOfSoupULreviews = SoupIDreviews.find_all("ul", {"class": "undefined list__09f24__ynIEd"})

In [7]:
SoupListReviews = ListOfSoupULreviews[0].find_all("li")

In [8]:
#This is a good reason for find all as each person's review is a separate li element
for SoupReview in SoupListReviews:
    #If mutiple people have responded there can be more than one a tag, 
    #We assume that the first one is the person making the review
    Person = SoupReview.find("a", {"class": "css-1m051bw"})
    print("Review User ID:", Person['href'].split("userid=")[-1])
    print("  " + Person['href'])
    print("  " + Person.text.strip()) # strip removes any HTML tags
    
    #if you want more information about responses to review you might want mutiple dates
    ReviewDateList = SoupReview.find_all("span", {"class": "css-chan6m"})
    
    #Different ways you might want/need to deal with date/times
    #How a website handles timezones is something you might want to experiement with too.
    print("  " + ReviewDateList[0].text.strip())
    varReviewDate = dateparser.parse(ReviewDateList[0].text.strip())
    print("  ", varReviewDate, type(varReviewDate), varReviewDate.date(), type(varReviewDate.date()))
    
    #depending on the rating the other classes in this div can change so only 1 was used that *should* be in all of them
    SoupRating = SoupReview.find("div", {"class": "five-stars__09f24__mBKym"})
    print(SoupRating["aria-label"])
    print(SoupRating["aria-label"][0])
    
    #a lot of investigation went into this you can figure out the raving from the number of path results
    SVGPathList = SoupRating.find_all("path", {"opacity": "1"})
    
    print("    Expected Rating out of 10 from SVG: ", len(SVGPathList))
    for Path in SVGPathList:
        print("     Rating Color - " + Path['fill'])
        
    for ReviewText in SoupReview.find_all("p"):
        print("------------------------------")
        print(ReviewText.text.strip())
    print("===============================")
    

Review User ID: XNhutnn3jTwrTS657q5bKw
  /user_details?userid=XNhutnn3jTwrTS657q5bKw
  Rachael H.
  12/14/2022
   2022-12-14 00:00:00 <class 'datetime.datetime'> 2022-12-14 <class 'datetime.date'>
5 star rating
5
    Expected Rating out of 10 from SVG:  10
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
     Rating Color - rgba(251,67,60,1)
------------------------------
10/10 definitely recommend. The staff is so friendly. The dentists are very thorough and they have all the newest technology. My husband, daughter, and I trust Enamel with all of our dental needs!
Review User ID: e-uQEcLyXN6G1gbgZHZ85g
  /user_details?userid=e-uQEcLyXN6G1gbgZHZ85g
  Holly J.
  12/13/2022
   2022-12-13 00:00:00 <cl

In [9]:
#Cleanup the URL so it can be a file name
filename = response.url.replace("https://www.yelp.com/biz/", "")
filename = filename.replace("/", "_")
filename = filename.replace(":", " ")

filename = filename.replace("=", "")
filename = filename.replace("?", " ")


# If you are going to write the response out to a file ALWAYS use prettify it makes it possible to read it as a human.
with open(filename + ".html", "w") as f:
    f.write(SoupFullPage.prettify())

In [10]:
SoupPagination = SoupFullPage.find("div", {"class": "pagination__09f24__VRjN4"})

print("Page navigation info")
print(SoupPagination.prettify())
print("-------------------")


#Get the first Div tag
NavigateSiblings = SoupPagination.div
print("nav Siblings (same hiearchy level)")
print(NavigateSiblings.prettify())
print("-------------------")

#Get the net sibling
PageInfoDiv = NavigateSiblings.find_next_sibling("div")

print(PageInfoDiv.prettify())
print("-------------------")

print(PageInfoDiv.text.strip())

CurrentPage_LastPage = PageInfoDiv.text.strip().split(" of ")

Page navigation info
<div aria-label="Pagination navigation" class="pagination__09f24__VRjN4 border-color--default__09f24__NPAKY" role="navigation">
 <div class="border-color--default__09f24__NPAKY">
  <div class="pagination-links__09f24__bmFj8 border-color--default__09f24__NPAKY nowrap__09f24__lBkC2">
   <div class="navigation-button-container__09f24__SvcBh border-color--default__09f24__NPAKY">
    <span class="css-foyide">
     <span alt="" aria-hidden="true" class="icon--24-chevron-left-v2 navigation-button-icon__09f24__Bmrde navigation-button-icon--disabled__09f24__z98Q4 css-1lx34my" role="img">
      <svg class="icon_svg" height="24" width="24">
       <path d="M14.25 17.58a1 1 0 01-.71-.3L9 12.7a1 1 0 010-1.4l4.5-4.58A1 1 0 0115 6.7a1 1 0 010 1.42L11.15 12 15 15.88a1 1 0 010 1.42 1 1 0 01-.75.28z">
       </path>
      </svg>
     </span>
    </span>
   </div>
   <div class="pagination-link-container__09f24__RAlwO border-color--default__09f24__NPAKY">
    <span aria-current="true

In [11]:
print(CurrentPage_LastPage)

['1', '20']


In [12]:
def GetAndParseYelpReviewURL(url, saveHTML, TimesToTryAgain=3):
    #This will be the object this function returns
    ListDictResults = []
    
    #Try to go to the URL
    response = requests.get(url)
    
    #Check for a return status of 200 or repeat waiting 10 seconds between requests until try again is reached
    webpagefailures = 0
    while response.status_code != 200:
        time.sleep(10)
        response = requests.get(url)
        webpagefailures = webpagefailures + 1
        
        if webpagefailures > TimesToTryAgain:
            return "FAILURE"
    
    #Start parsing the html of the page:
    SoupFullPage = BeautifulSoup(response.text, 'lxml')
    
    # If you have selected to save the page then write it out to a file
    if saveHTML:
        #Cleanup the URL so it can be a file name
        filename = response.url.replace("https://www.yelp.com/biz/", "")
        filename = filename.replace("/", "_")
        filename = filename.replace(":", " ")

        filename = filename.replace("=", "")
        filename = filename.replace("?", " ")
        
        with open(filename + ".html", "w") as f:
            f.write(SoupFullPage.prettify())
    
    
    SoupIDreviews = SoupFullPage.find("div", {"id": "reviews"})
    ListOfSoupULreviews = SoupIDreviews.find_all("ul", {"class": "undefined list__09f24__ynIEd"})
    
    SoupListReviews = ListOfSoupULreviews[0].find_all("li")
    
    for SoupReview in SoupListReviews:
        ReviewDictionary = {}
        
        Person = SoupReview.find("a", {"class": "css-1m051bw"})
        
        ReviewDictionary['PersonID'] = Person['href'].split("userid=")[-1]
        ReviewDictionary['PersonURLpart'] = Person['href']
        ReviewDictionary['PersonName'] = Person.text.strip()
        
        #Only Getting the first date
        ReviewDateElement = SoupReview.find("span", {"class": "css-chan6m"})

        #If not storing in json better to use native python date
        #ReviewDictionary['ReviewDate'] = dateparser.parse(ReviewDateElement.text.strip())
        revdate = dateparser.parse(ReviewDateElement.text.strip())
        ReviewDictionary['ReviewDate'] = revdate.strftime('%Y-%m-%d')
                                   
        #rating                                                  
        SoupRating = SoupReview.find("div", {"class": "five-stars__09f24__mBKym"})
        
        ReviewDictionary['Rating'] = int(SoupRating["aria-label"][0])
        
        AllReviewText = ""
        for ReviewText in SoupReview.find_all("p"):
            print("------------------------------")
            AllReviewText = AllReviewText + " " + ReviewText.text.strip()
        ReviewDictionary['ReviewText'] = AllReviewText
                                                          
        ListDictResults.append(ReviewDictionary)

    return ListDictResults

In [13]:
# For yelp in the address bar pages are repersented by the get variable &start=
# In our case page 1 does not have this page 2 is 10 page 3 is 20 etc

BusinessID = 12345
AllBusinessReviews = []
#Note this only records pages 2 and on to the dictionary
for i in range(2, int(CurrentPage_LastPage[1])+1):
    
    
    print("Page: ", i)
    print("start var: ", (i-1)*10)
    
    if "?" in BaseUrl:
        print("Already get variables just add to the end")
        NextURL = BaseUrl + "&start=" + str((i-1)*10)
    else:
        NextURL = BaseUrl + "?start=" + str((i-1)*10)
    
    #Get and Parse the next page of reviews
    CurrentPageReviews = GetAndParseYelpReviewURL(NextURL, saveHTML=True, TimesToTryAgain=3)
    AllBusinessReviews.extend(CurrentPageReviews)
    #for safty if the script fails export the current info to a json file every 10 pages
    if i%3 == 0:
        with open("ReviewBackup" + str(BusinessID) + ".json", "w") as outfile:
            json.dump({BusinessID: AllBusinessReviews, 'TotalReviews': len(AllBusinessReviews), 'PageNum': i}, outfile, indent=4)
        
with open(str(BusinessID) + ".json", "w") as outfile:
    json.dump({BusinessID: AllBusinessReviews, 'TotalReviews': len(AllBusinessReviews)}, outfile, indent=4)
        

Page:  2
start var:  10
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
Page:  3
start var:  20
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
Page:  4
start var:  30
-----------------------------

In [14]:
print(json.dumps({
    BusinessID: AllBusinessReviews, 
    'TotalReviews': len(AllBusinessReviews)
              }, indent=4))

{
    "12345": [
        {
            "PersonID": "jjXRlB6ae4y7GiGWfbb6Jw",
            "PersonURLpart": "/user_details?userid=jjXRlB6ae4y7GiGWfbb6Jw",
            "PersonName": "Michele M.",
            "ReviewDate": "2021-05-01",
            "Rating": 5,
            "ReviewText": " Time to give these legends another glowing review. \u00a0 Had an unexpected dental catastrophe right before I was suppose to get my 2nd covid shot. \u00a0The staff were able to squeeze me in the next morning and I was sent to a e specialists for a further scan. \u00a0Turned out I my tooth was super infected and an ancient root canalHad given up doing it's job. \u00a0 I find this all put Friday afternoon and get prescribed antibiotics. \u00a07pm I'm feeling like my inflammation is out of control so I text dr Dillon and he says he will come into work on Saturday and extract the tooth. So here I am Saturday afternoon. With my tooth extracted and a lot leas pain and anguish. \u00a0 I absolutely love these guy